In [5]:
!pip install bbpylib
from bbpylib.colab.tools import require_gdrive
require_gdrive()

Hello this is an update of bbpylib
Mounted at /content/drive


In [11]:
from __future__ import annotations

from pathlib import Path
import ipywidgets as w
from IPython.display import display

try:
    from ipyfilechooser import FileChooser
except ImportError as e:
    raise ImportError(
        "ipyfilechooser is not installed.\n"
        "In Colab run:\n"
        "  !pip -q install ipyfilechooser\n"
    ) from e


class FileChooserEditor:
    """
    Combined FileChooser + Editor for Colab.

    Workflow:
    - Choose a file under `root_dir` and click the FileChooser's Select button.
    - Load button turns green/enabled when:
        * a file is selected AND
        * either no file is loaded yet, or the selected file differs from the loaded file AND
        * there are no unsaved edits in the current file.
    - If current file is dirty (unsaved), loading another file is blocked until Save or Quit.
    """

    def __init__(
        self,
        root_dir: str | Path,
        *,
        filter_pattern: str = "*.py",
        editor_height: str = "520px",
        path_chars: int = 90,
    ):
        self.root_dir = Path(root_dir).expanduser().resolve()
        if not self.root_dir.is_dir():
            raise NotADirectoryError(self.root_dir)

        # ---- state ----
        self.current_path: Path | None = None   # loaded into editor
        self.pending_path: Path | None = None   # selected in chooser (candidate to load)
        self._baseline_text: str = ""
        self._dirty: bool = False
        self._path_chars = path_chars

        # ---- chooser ----
        self.fc = FileChooser(str(self.root_dir), select_desc="Select")
        self.fc.filter_pattern = filter_pattern
        self.fc.title = "Choose file"
        self.fc.register_callback(self._on_chosen)

        # ---- editor ----
        self.editor = w.Textarea(layout=w.Layout(width="100%", height=editor_height))
        self.editor.add_class("monospace")
        self.editor.observe(self._on_editor_change, names="value")

        # ---- UI ----
        self.path_text = w.HTML("<code>(no file loaded)</code>")
        self.status = w.Label("")

        self.load_button = w.Button(description="Load", disabled=True, button_style="")
        self.load_button.on_click(self._on_load_click)

        self.save_button = w.Button(description="Saved", button_style="success", disabled=True)
        self.save_button.on_click(self._on_save_click)

        self.quit_button = w.Button(description="Quit", button_style="success", disabled=True)
        self.quit_button.on_click(self._on_quit_click)

        self.ui = w.VBox(
            [
                self.fc,
                w.HBox([self.load_button, self.save_button, self.quit_button]),
                self.path_text,
                self.status,
                self.editor,
            ],
            layout=w.Layout(width="100%"),
        )

        self._update_controls()

    def display(self):
        display(self.ui)

    # ---------------- helpers ----------------

    def _short_path_html(self, p: Path | None) -> str:
        if p is None:
            return "<code>(no file loaded)</code>"
        s = str(p)
        shown = ("…" + s[-self._path_chars:]) if len(s) > self._path_chars else s
        return f"<code>{shown}</code>"

    def _set_dirty(self, dirty: bool):
        self._dirty = dirty
        if dirty:
            self.save_button.description = "Save"
            self.save_button.button_style = "warning"  # orange
            self.quit_button.button_style = "warning"
        else:
            self.save_button.description = "Saved"
            self.save_button.button_style = "success"  # green
            self.quit_button.button_style = "success"

    def _refresh_pending_from_fc(self):
        sel = getattr(self.fc, "selected", None)
        if not sel:
            self.pending_path = None
            return

        p = Path(sel).expanduser().resolve()

        # enforce root
        try:
            p.relative_to(self.root_dir)
        except ValueError:
            self.pending_path = None
            self.status.value = "Selection outside root folder (blocked)."
            return

        # only allow files
        if p.exists() and p.is_dir():
            self.pending_path = None
            self.status.value = "Selected item is a folder; please select a file."
            return

        self.pending_path = p

    def _update_controls(self):
        loaded = self.current_path is not None
        self.save_button.disabled = not loaded
        self.quit_button.disabled = not loaded

        if self.pending_path is None:
            self.load_button.disabled = True
            self.load_button.button_style = ""  # grey
            return

        # If selected file is already loaded, do not show "ready to load".
        if self.current_path is not None and self.pending_path == self.current_path:
            self.load_button.disabled = True
            self.load_button.button_style = ""
            self.status.value = f"Selected (already loaded): {self.pending_path.name}"
            return

        # Selected file differs from loaded file
        if self._dirty:
            self.load_button.disabled = True
            self.load_button.button_style = ""
            self.status.value = "Unsaved changes: Save or Quit before loading another file."
            return

        self.load_button.disabled = False
        self.load_button.button_style = "success"  # green
        self.status.value = f"Ready to load: {self.pending_path.name}"

    # ---------------- events ----------------

    def _on_chosen(self, *args, **kwargs):
        # ipyfilechooser callback signature varies; always read self.fc.selected
        try:
            self._refresh_pending_from_fc()
            self._update_controls()
        except Exception as e:
            self.status.value = f"Select callback error: {e}"

    def _on_load_click(self, _):
        if self.pending_path is None:
            self.status.value = "No file selected (click Select in the chooser first)."
            return
        if self._dirty:
            self.status.value = "Blocked: current file has unsaved changes."
            self._update_controls()
            return

        try:
            text = self.pending_path.read_text(encoding="utf-8")
        except Exception as e:
            self.status.value = f"Load failed: {e}"
            return

        # set editor without triggering dirty flicker
        self.editor.unobserve(self._on_editor_change, names="value")
        self.editor.value = text
        self.editor.observe(self._on_editor_change, names="value")

        self.current_path = self.pending_path
        self._baseline_text = text
        self._set_dirty(False)
        self.path_text.value = self._short_path_html(self.current_path)
        self.status.value = f"Loaded: {self.current_path.name}"
        self._update_controls()

    def _on_editor_change(self, change):
        if self.current_path is None:
            return
        self._set_dirty(change["new"] != self._baseline_text)
        self._update_controls()

    def _on_save_click(self, _):
        if self.current_path is None:
            return
        try:
            self.current_path.write_text(self.editor.value, encoding="utf-8")
        except Exception as e:
            self.status.value = f"Save failed: {e}"
            return

        self._baseline_text = self.editor.value
        self._set_dirty(False)
        self.status.value = f"Saved: {self.current_path.name}"
        self._update_controls()

    def _on_quit_click(self, _):
        # Quit = close current file and clear editor; pending selection remains.
        self.current_path = None
        self._baseline_text = ""
        self._set_dirty(False)

        self.editor.unobserve(self._on_editor_change, names="value")
        self.editor.value = ""
        self.editor.observe(self._on_editor_change, names="value")

        self.path_text.value = self._short_path_html(None)
        self.status.value = "Quit: no file loaded."
        self._update_controls()


In [12]:
import os
os.getcwd()
fce = FileChooserEditor("/content/drive/MyDrive/GIT-repos", filter_pattern="*.py")
fce.display()